In [1]:
import torch
import sys

# Add the parent directory of the script (i.e., project/) to sys.path
sys.path.append('../')

from utils.bvp import BVP
from utils.upinn import UPINN
from utils.DataGenerators import LotkaVolterra, sample_with_noise
from utils.architectures import FNN

### Generate Data

In [2]:
###############################################
### Generate data from Lotka-Volterra model ###
###############################################
###   dx/dt = alpha*x - beta*x*y            ###
###   dy/dt = gamma*x*y - delta*y           ###
###############################################
alpha, beta, gamma, delta = 2/3, 4/3, 1.0, 1.0
x0, y0 = 1.0, 1.0
LV = LotkaVolterra(alpha, beta, gamma, delta, torch.tensor([x0, y0], dtype=torch.float32))

time_int = [0, 25]
train_test = 0.8
N = 800
t = torch.linspace(time_int[0], time_int[1], N)
X = LV.solve(t)
train_idx = torch.arange(0, train_test*N, dtype=torch.long)
test_idx = torch.arange(train_test*N, N, dtype=torch.long)

# Sample subset and add noise
t_d, X_d = sample_with_noise(10, t[train_idx], X[train_idx], epsilon=5e-3)

### Setup Boundary Value Problem

In [3]:
class LV_BVP(BVP):
    
    def __init__(self, params):
        super().__init__(params)

    def f(self, z, U):
        alpha = self.params['alpha'] if 'alpha' in self.params else self.alpha
        beta = self.params['beta'] if 'beta' in self.params else self.beta
        delta = self.params['delta'] if 'delta' in self.params else self.delta

        dUdt = torch.cat([
        torch.autograd.grad(outputs=U[:, i], inputs=z, grad_outputs=torch.ones_like(U[:, i]), create_graph=True)[0]
        for i in range(U.shape[1])
        ], dim=-1)

        return torch.stack([
            dUdt[:, 0] - alpha*U[:, 0] + beta*U[:, 0]*U[:, 1],
            dUdt[:, 1] + delta*U[:, 1] # - gamma*U[:, 0]*U[:, 1] <-- Estimate this
        ], dim=-1)
    

    def g(self, z, U):
        return U - 1.0

### Setup UPINN

In [4]:
# Define model architectures
hidden = [16] * 4
u = FNN(
    dims=[1, *hidden, 2],
    hidden_act=torch.nn.Tanh(),
    output_act=torch.nn.Softplus(),
)
G = FNN(
    dims=[2, *hidden, 2],
    hidden_act=torch.nn.Tanh(),
    output_act=torch.nn.ReLU(),
)

# Setup scaling layer
u.scale_fn = lambda t_: (t_-t.min())/(t.max()-t.min())
mu, sigma = 0, 2
epsilon = 1e-8
G.scale_fn = lambda x: (x-mu)/(sigma+epsilon)

# Define model
# params = dict(
#     alpha=torch.nn.Parameter(torch.tensor(0.5)),
#     beta=torch.nn.Parameter(torch.tensor(0.5)),
#     delta=torch.nn.Parameter(torch.tensor(0.5))
# )
params = dict(
    alpha=alpha,
    beta=beta,
    delta=delta,
)


upinn = UPINN(u, G, bvp=LV_BVP(params))

In [5]:
import plotly.graph_objects as go

class LV_Plotter:
    def __init__(self, t, X, t_d, X_d, gamma):
        self.t = t
        self.X = X
        self.z_d = t_d
        self.U_d = X_d
        self.gamma = gamma

    def __call__(self, u, G):
        device = next(u.parameters()).device # Get the device of the model
        plots = dict()

        # Evaluate the model
        X_pred = u(self.t.unsqueeze(-1).to(device))

        # Plot with plotly
        fig = go.Figure()
        fig.add_scatter(x=self.t.cpu().numpy(), y=self.X[:, 0].cpu().numpy(), mode='lines', name='Prey', line=dict(dash='dash', color='green'))
        fig.add_scatter(x=self.t.cpu().numpy(), y=self.X[:, 1].cpu().numpy(), mode='lines', name='Predator', line=dict(dash='dash', color='red'))
        fig.add_scatter(x=self.t.cpu().numpy(), y=X_pred[:, 0].cpu().numpy(), mode='lines', name='Prey (pred)', line=dict(color='green'))
        fig.add_scatter(x=self.t.cpu().numpy(), y=X_pred[:, 1].cpu().numpy(), mode='lines', name='Predator (pred)', line=dict(color='red'))
        # Add datapoints
        fig.add_scatter(x=self.z_d.squeeze().cpu().numpy(), y=self.U_d[:, 0].cpu().numpy(), mode='markers', name='Prey (data)', marker=dict(color='green', symbol='x'))
        fig.add_scatter(x=self.z_d.squeeze().cpu().numpy(), y=self.U_d[:, 1].cpu().numpy(), mode='markers', name='Predator (data)', marker=dict(color='red', symbol='x'))
        fig.update_layout(title=f"Lotka-Volterra Model")
        
        # Log figure to wandb
        plots["Solution"] = fig

        # Plot missing terms
        res = G(X_pred).cpu()
        res_dx = res[:, 0]
        res_dy = res[:, 1]
        true_res_dx = torch.zeros_like(res_dx)
        true_res_dy = (self.gamma*self.X[:, 0]*self.X[:, 1]).cpu().numpy()

        fig = go.Figure()
        fig.add_scatter(x=self.t.cpu().numpy(), y=res_dx, mode='lines', name='Residual Prey', line=dict(color='green'))
        fig.add_scatter(x=self.t.cpu().numpy(), y=res_dy, mode='lines', name='Residual Predator', line=dict(color='red'))
        fig.add_scatter(x=self.t.cpu().numpy(), y=true_res_dx, mode='lines', name='Prey: 0', line=dict(dash='dash', color='green'))
        fig.add_scatter(x=self.t.cpu().numpy(), y=true_res_dy, mode='lines', name='Predator: γ*x*y', line=dict(dash='dash', color='red'))
        fig.update_layout(title=f"Lotka-Volterra Missing Terms")

        # Log figure to wandb
        plots["Missing Terms"] = fig

        return plots

plotter = LV_Plotter(t, X, t_d, X_d, gamma=gamma)

In [6]:
data_points = t_d.unsqueeze(-1)
data_target = X_d

boundary_points = torch.zeros(1, 1)

collocation_points = t.unsqueeze(-1).requires_grad_(True)

upinn.train(
    data_points,
    data_target,
    boundary_points,
    collocation_points,
    epochs=10000,
    log_wandb=dict(name='UPINN', project='Master-Thesis', plotter=plotter, plot_interval=1000),
    optimizer=torch.optim.AdamW,
    optimizer_args=dict(lr=0.0028621743872566546, weight_decay=0.00000000034746604191),
    beta_softadapt=0.0,
    # scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau,
    # scheduler_args=dict(factor=0.5, patience=100, min_lr=1e-6),
    )

Beginning training...
Running on: cuda


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: megajosni. Use `wandb login --relogin` to force relogin
wandb: WARNING Path <TemporaryDirectory 'C:\\Users\\jonas\\AppData\\Local\\Temp\\tmp868prn7c'>\wandb\ wasn't writable, using system temp directory.
